# **Automatic Summarization System**

Authors: Manuel Casas, Bárbara Escalante & Tomás Mermelstein

# Imports

In [ ]:
# Connect to google drive
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pdfplumber sentence-transformers faiss-cpu transformers pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

In [ ]:
import re
import pdfplumber
import re
import pdfplumber
import json

# Divide the full PDF into smaller chunks

We are going to define a function that, given a PDF, it divides into its different 'Artículos':

In [ ]:
def extract_articulos_to_dict(pdf_path):
    # Regex to match "Artículo X." or "Artículo X" (with optional punctuation)
    # Added handling for numbers followed by "." or other separators
    articulo_pattern = re.compile(r'Artículo\s+(\d+)[\.\s]')  # Matches "Artículo 2." or "Artículo 2 "
    articulos_dict = {}
    current_articulo = None
    current_text = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue

            # Split text into lines to handle multi-line headers
            lines = text.split('\n')
            for line in lines:
                # Check for "Artículo X" in the line
                match = articulo_pattern.match(line.strip())  # Use .match() for start-of-line
                if match:
                    # Save previous article (if exists)
                    if current_articulo:
                        articulos_dict[current_articulo] = "\n".join(current_text)
                        current_text = []

                    # Start new article
                    articulo_number = match.group(1)
                    current_articulo = f"Artículo {articulo_number}"

                    # Add the rest of the line (after "Artículo X") to the current article
                    remaining_text = line.split(match.group(0), 1)[-1].strip()
                    if remaining_text:
                        current_text.append(remaining_text)
                else:
                    # Append to current article
                    if current_articulo:
                        current_text.append(line.strip())

    # Add the last article
    if current_articulo and current_text:
        articulos_dict[current_articulo] = "\n".join(current_text)

    return articulos_dict

# Divide articles depending on their information.

In this approach, the data is organized into a JSON file. The extracted information focuses on the aspects most relevant to scholarship applcants, and it is categorized into three main sections: general information, university students, and non-university students. Each of these sections is further subdivided into subcategories such as requirements, scholarship amounts, and deadlines. To ensure the best possible results when generating the summary, all these sections are presented in Spanish, as the original document (PDF) is in Spanish, and this language choice directly reflects the extracted data.

With the assistance of ChatGPT and a personal review to verify the proposed structure, the articles were effectively organized into their respective sections and subsections, as outlined above.

In [ ]:
def structure_data(articulos_dict):
    # Base dictionary for the structured JSON output
    structured_data = {
        "general": {
            "requisitos": [],
            "cuantias": [],
            "plazos": []
        },
        "universitarios": {
            "requisitos": [],
            "cuantias": [],
            "obligaciones": []
        },
        "no_universitarios": {
            "requisitos": [],
            "cuantias": [],
            "obligaciones": []
        }
    }

    # Mapping of articles to categories (based on prior knowledge)
    article_mapping = {
        "general": {
            "requisitos": ["1", "15", "16", "17", "18", "19", "20"],
            "cuantias": ["4", "5", "6", "7", "8", "9", "11", "12"],
            "plazos": ["47", "48"]
        },
        "universitarios": {
            "requisitos": ["22", "23", "24", "28"],
            "cuantias": ["5", "13"],
            "obligaciones": ["40", "43"]
        },
        "no_universitarios": {
            "requisitos": ["32", "33", "34", "35"],
            "cuantias": ["9", "12"],
            "obligaciones": ["40", "43"]
        }
    }

    # Assign content to appropriate categories
    for articulo, content in articulos_dict.items():
        articulo_num = articulo.split()[1]  # Extract the article number

        # Find which category this article belongs to
        for category, sections in article_mapping.items():
            for section, articles in sections.items():
                if articulo_num in articles:
                    # Basic content cleaning
                    cleaned_content = re.sub(r'\s+', ' ', content).strip()
                    if cleaned_content:
                        structured_data[category][section].append({
                            "articulo": articulo,
                            "contenido": cleaned_content
                        })

    return structured_data


In [ ]:
pdf_path = "/content/drive/MyDrive/APLN/ayudas_24-25.pdf"
articulos_dict = extract_articulos_to_dict(pdf_path)
structured_json = structure_data(articulos_dict)

# Save in JSON
with open("/content/drive/MyDrive/APLN/structured_scholarships.json", "w", encoding="utf-8") as f:
    json.dump(structured_json, f, ensure_ascii=False, indent=4)

print(json.dumps(structured_json, indent=2, ensure_ascii=False))

{
  "general": {
    "requisitos": [
      {
        "articulo": "Artículo 1",
        "contenido": "Objeto y beneficiarios. 1. Se convocan por la presente Resolución becas para estudiantes que en el curso académico 2024-2025, cursen enseñanzas postobligatorias con validez en todo el territorio nacional. 2. Para ser beneficiario de las becas que se convocan por esta Resolución será preciso cumplir los requisitos básicos establecidos en el Real Decreto 1721/2007, de 21 de diciembre, así como los que se fijan en el Real Decreto 201/2024, de 27 de febrero, por el que se establecen los umbrales de renta y patrimonio familiar y las cuantías de las becas y ayudas al estudio del Ministerio de Educación, Formación Profesional y Deportes para el curso 2024-2025 y en esta convocatoria. 3. Tendrán la consideración de beneficiarios de las becas a las que se refiere la presente convocatoria, los estudiantes a quienes se les concedan en cualquiera de los siguientes supuestos: a) Que perciban directa

Using the extracted JSON data, a summary was generated with the help of ChatGPT. The JSON file was provided to the language model, and various prompts were tested to achieve the most accurate result. The best outcome was obtained by instructing the model to create a summary while preserving the structure of the original structured file.

Upon reviewing the output against the full document, it was found that certain monetary amounts and academic requirements were missing from the summary. The main hypothesis for this issue is the large volume of data being passed to the language model, each article is submitted in its entirety. To address this, regular expression (Regex) rules were implemented to filter and extract the most relevant information from the articles before summarization.

## Filter the articles with Reges rules

The most clearly identifiable elements for filtering are the data presented in lists and tables (such as *umbrales*), as well as relevant dates. In addition to these, certain keywords play an important role. During the PDF parsing process, some CSV-format data was also extracted—this includes numerical values considered relevant for filtering, as well as URLs and inverted terms. Based on this, a set of rules was established to filter the content of the articles, focusing on key patterns such as lists, tables, and dates.

Below is the complete code used to extract structured data from the BOE PDF document and generate a summarization from that information.

In [ ]:
def extract_articulos_to_dict(pdf_path):
    # Regex to match "Artículo X." or "Artículo X" (with optional punctuation)
    # Added handling for numbers followed by "." or other separators
    articulo_pattern = re.compile(r'Artículo\s+(\d+)[\.\s]')  # Matches "Artículo 2." or "Artículo 2 "
    articulos_dict = {}
    current_articulo = None
    current_text = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue

            # Split text into lines to handle multi-line headers
            lines = text.split('\n')
            for line in lines:
                # Check for "Artículo X" in the line
                match = articulo_pattern.match(line.strip())  # Use .match() for start-of-line
                if match:
                    # Save previous article (if exists)
                    if current_articulo:
                        articulos_dict[current_articulo] = "\n".join(current_text)
                        current_text = []

                    # Start new article
                    articulo_number = match.group(1)
                    current_articulo = f"Artículo {articulo_number}"

                    # Add the rest of the line (after "Artículo X") to the current article
                    remaining_text = line.split(match.group(0), 1)[-1].strip()
                    if remaining_text:
                        current_text.append(remaining_text)
                else:
                    # Append to current article
                    if current_articulo:
                        current_text.append(line.strip())

    # Add the last article
    if current_articulo and current_text:
        articulos_dict[current_articulo] = "\n".join(current_text)

    return articulos_dict

def structure_data(articulos_dict):
    # Base dictionary for the structured JSON output
    structured_data = {
        "general": {
            "requisitos": [],
            "cuantias": [],
            "plazos": []
        },
        "universitarios": {
            "requisitos": [],
            "cuantias": [],
            "obligaciones": []
        },
        "no_universitarios": {
            "requisitos": [],
            "cuantias": [],
            "obligaciones": []
        }
    }

    # Mapping of articles to categories (based on prior knowledge)
    article_mapping = {
        "general": {
            "requisitos": ["1", "15", "16", "17", "18", "19", "20"],
            "cuantias": ["4", "5", "6", "7", "8", "9", "11", "12"],
            "plazos": ["47", "48"]
        },
        "universitarios": {
            "requisitos": ["22", "23", "24", "28"],
            "cuantias": ["5", "13"],
            "obligaciones": ["40", "43"]
        },
        "no_universitarios": {
            "requisitos": ["32", "33", "34", "35"],
            "cuantias": ["9", "12"],
            "obligaciones": ["40", "43"]
        }
    }

    # Assign content to appropriate categories
    for articulo, content in articulos_dict.items():
        articulo_num = articulo.split()[1]  # Extract the article number

        # Find which category this article belongs to
        for category, sections in article_mapping.items():
            for section, articles in sections.items():
                if articulo_num in articles:
                    # Basic content cleaning
                    cleaned_content = re.sub(r'\s+', ' ', content).strip()
                    if cleaned_content:
                        structured_data[category][section].append({
                            "articulo": articulo,
                            "contenido": cleaned_content
                        })

    return structured_data

def contains_reversed_text(texto, threshold=3):
    """
    Detects if a line contains reversed Spanish text
    by comparing against common reversed words.
    """
    reversed_words = [
        'ogidóC', 'nóiccerid', 'otnemucod', 'dadirgetni', 'racifirev', 'oruges',
        'se.bog.noicartsinimda', 'edes', 'etneiugis', 'al', 'ne', 'etse', 'ed', 'edeuP'
    ]

    counter = 0
    for word in reversed_words:
        if word in texto:
            counter += 1
            if counter >= threshold:
                return True

    # Check for NEG- code (reversed GEN-)
    if 'NEG-' in texto:
        return True

    return False

def filter_content(texto):
    # Patterns to completely exclude
    exclusion_patterns = [
        # Codes and validations
        r'.*GEN-[\w-]{36}.*',  # Lines with GEN codes
        r'.*CSV\s*:\s*GEN-[\w-]+.*',  # Any line with "CSV: GEN-"
        r'.*DIRECCIÓN DE VALIDACIÓN\s*:\s*https?://\S+.*',  # Lines with validation URL
        r'.*código seguro de verificación.*',
        r'.*nóicacifireV.*',

        # Lines starting with number followed by CSV
        r'^\d+\s+CSV\s*:.*',  # Lines starting with number and CSV

        # URLs and links
        r'.*https?://\S+.*',

        # Signatures and metadata
        r'.*FIRMANTE\(\d+\)\s*:.*FECHA\s*:.*Aprueba.*',   # Lines with signer and date
        r'.*Aprueba\s+\.\.\..*',
        r'.*\|.*\|.*',  # Lines with multiple pipes

        # Other administrative content
        r'.*[a-z]{4,}\s*:\s*GEN-[a-z0-9-]+.*',

        # Pattern for long lines with multiple administrative elements
        r'^\d+\s+CSV\s*:\s*GEN-[\w-]+\s+DIRECCIÓN DE VALIDACIÓN\s*:.*FIRMANTE.*FECHA.*Aprueba.*',
        r'.*\b[a-f0-9]{4,}-[a-f0-9]{4,}-[a-f0-9]{4,}-[a-f0-9]{4,}-[a-f0-9]{4,}-[a-f0-9]{4,}-[a-f0-9]{4,}-NEG\b.*',
        r'^\d+\s*[:|]?\s*\|?\s*:?\s*$',
        r'^[\s:|.\-–—·•]{2,}$'

    ]

    # Specific words that appear in reversed text
    reversed_words = [
        'ogidóC', 'nóiccerid', 'otnemucod', 'dadirgetni', 'racifirev', 'oruges',
        'se.bog.noicartsinimda', 'edes', 'etneiugis', 'al', 'ne', 'etse', 'ed', 'edeuP'
    ]

    # More comprehensive pattern for reversed validation codes
    exclusion_patterns.extend([
        r'.*NEG-[\w-]+.*',  # Any line with NEG- (reversed GEN-)
        r'.*[\d]+\s*:\s*.*NEG-.*',  # Lines with number and NEG- code
        r'.*\|\s*.*NEG-.*',  # Lines with pipe and NEG-
        r'^.*al ne etse.*$',  # Specific reversed phrases
        r'^.*edeuP.*$',  # Lines with "edeuP" (reversed "Puede")
    ])

    # Patterns to keep even if they contain exclusion elements
    keep_patterns = [
        r'\bThreshold \d+ \(euros\)',
        r'\d+,\d+ euros',
        r'\b\d+ credits\b',
        r'\b\d+%\b',
        r'\b\d+,\d+\b',
        r'\b\d+\s*-\s*\d+\b'
    ]

    lines = texto.split('\n')
    filtered_lines = []
    in_list = False
    current_list = []
    list_pattern = re.compile(r'^\s*(\d+\.\s+|[a-zA-Z]\)\s+|[-•]\s+)')

    for line in lines:
        line = line.strip()

        if not line:
            continue

        if contains_reversed_text(line):
            continue

        # First check if there are patterns to keep
        should_keep = any(re.search(p, line) for p in keep_patterns)

        # Check if it should be excluded
        should_exclude = any(re.search(p, line) for p in exclusion_patterns)

        # Specifically check if it contains reversed text
        has_reversed_text = any(word in line for word in reversed_words)

        # If it contains a pattern to keep, we keep it regardless
        if should_keep:
            filtered_lines.append(line)
            continue

        # If it has an exclusion pattern or reversed text, we skip it
        if should_exclude or has_reversed_text:
            continue

        # Handle lists
        is_list_item = list_pattern.match(line)
        if is_list_item:
            if not in_list:
                in_list = True
                current_list = [line]
            else:
                current_list.append(line)
        else:
            if in_list:
                # Finish the current list
                filtered_lines.extend(current_list)
                current_list = []
                in_list = False

            # Add the current line if we got here
            filtered_lines.append(line)

    # Handle a list at the end of the text
    if in_list and current_list:
        filtered_lines.extend(current_list)

    filtered_content = '\n'.join(filtered_lines)

    # Final cleanup
    filtered_content = re.sub(r'\n{3,}', '\n\n', filtered_content)  # Reduce whitespace
    filtered_content = re.sub(r'[•·\-–—]{2,}', '', filtered_content)  # Remove symbol repetitions
    filtered_content = re.sub(r'\s{2,}', ' ', filtered_content)  # Normalize spaces
    filtered_content = re.sub(r'^\s+$', '', filtered_content, flags=re.MULTILINE)  # Remove whitespace-only lines

    # Additional filter: Remove lines containing numbers followed by ".." (typical in verification codes)
    filtered_content = re.sub(r'^.*\d+\s*\.\.\..*$', '', filtered_content, flags=re.MULTILINE)

    # Filter lines containing "NEG-" (reversed GEN-)
    filtered_content = re.sub(r'^.*NEG-.*$', '', filtered_content, flags=re.MULTILINE)

    # Remove newline characters
    filtered_content = filtered_content.replace('\n', ' ')

    return filtered_content.strip()

def structure_data(articulos_dict):
    # Base dictionary for the structured JSON output
    structured_data = {
        "general": {
            "requisitos": [],
            "cuantias": [],
            "plazos": []
        },
        "universitarios": {
            "requisitos": [],
            "cuantias": [],
            "obligaciones": []
        },
        "no_universitarios": {
            "requisitos": [],
            "cuantias": [],
            "obligaciones": []
        }
    }

    # Mapping of articles to categories (based on prior knowledge)
    article_mapping = {
        "general": {
            "requisitos": ["1", "15", "16", "17", "18", "19", "20"],
            "cuantias": ["4", "5", "6", "7", "8", "9", "11", "12"],
            "plazos": ["47", "48"]
        },
        "universitarios": {
            "requisitos": ["22", "23", "24", "28"],
            "cuantias": ["5", "13"],
            "obligaciones": ["40", "43"]
        },
        "no_universitarios": {
            "requisitos": ["32", "33", "34", "35"],
            "cuantias": ["9", "12"],
            "obligaciones": ["40", "43"]
        }
    }

    # Assign content to appropriate categories and apply filtering
    for articulo, content in articulos_dict.items():
        articulo_num = articulo.split()[1]  # Extract the article number

        for category, sections in article_mapping.items():
            for section, articles in sections.items():
                if articulo_num in articles:
                    # Filter the content before adding it
                    filtered_content = filter_content(content)

                    if filtered_content:
                        structured_data[category][section].append({
                            "articulo": articulo,
                            "contenido": filtered_content
                        })

    return structured_data

In [ ]:
pdf_path = "/content/drive/MyDrive/APLN/ayudas_24-25.pdf"

articulos_dict = extract_articulos_to_dict(pdf_path)

structured_json = structure_data(articulos_dict)

with open("/content/drive/MyDrive/APLN/structured_scholarships.json", "w", encoding="utf-8") as f:
    json.dump(structured_json, f, ensure_ascii=False, indent=4)

print(json.dumps(structured_json, indent=2, ensure_ascii=False))

{
  "general": {
    "requisitos": [
      {
        "articulo": "Artículo 1",
        "contenido": "2024-2025, cursen enseñanzas postobligatorias con validez en todo el territorio nacional. Ministerio de Educación, Formación Profesional y Deportes para el curso 2024-2025 y en esta 1. Se convocan por la presente Resolución becas para estudiantes que en el curso académico convocatoria. supuestos: a) Que perciban directamente el importe. académico."
      },
      {
        "articulo": "Artículo 15",
        "contenido": "determinado nivel de conocimiento de un idioma extranjero. arquitecto técnico, ingeniero técnico o diplomado que haya obtenido la correspondencia con el ingeniero técnico o maestro que haya obtenido la correspondencia con el nivel 2 (grado) del Marco enumeran en el artículo 3 de esta Resolución. la obtención de la beca de matrícula. ascendientes directos a cargo y los descendientes directos a cargo menores de 21 años. En el supuesto de extranjeros no comunitarios, se ap

## Final summary with the filter data

The best prompt from before was used to generate the summary, the 'ayudas_24_25.pdf' was used, this way, the following results were obtained:

### ChatGPT

## 1. Información General
- **Plazos de solicitud:** Del 19 de marzo al 10 de mayo de 2024. Plazo excepcional hasta el 31 de diciembre en casos específicos.
- **Requisitos económicos:** Evaluación de renta y patrimonio familiar según umbrales establecidos.
- **Tipos de cuantías:** Becas de matrícula, renta, residencia, excelencia, cuantía variable y ayudas adicionales para transporte y familias monoparentales.

## 2. Requisitos Académicos
### Universitarios
- **Matrícula mínima:** Entre 30 y 59 créditos (matrícula parcial).
- **Créditos de beca:** Solo cubre créditos de primera matrícula.
- **Suspensos permitidos:** Aprobación del 40% de los créditos en carreras de ciencias y técnicas.
- **Nota media:** Se calcula sobre una escala de 0 a 10, con conversión de calificaciones cualitativas.

### No Universitarios
- **Asistencia mínima:** 80% salvo dispensa.
- **Aprobación mínima:** 50% de asignaturas o créditos matriculados.
- **Proyectos fin de estudios:** Plazo de un año para presentación.

## 3. Requisitos Económicos
- **Umbrales de renta según miembros de la familia:**
  - 1 miembro: 8.843€ (Umbral 1), 13.898€ (Umbral 2), 14.818€ (Umbral 3).
  - 4 miembros: 22.107€, 38.242€, 40.773€.
  - 8 miembros: 36.255€, 52.850€, 56.348€.
  - Miembros adicionales: Incremento de 3.536€ en Umbral 1, 3.340€ en Umbral 2 y 3.561€ en Umbral 3.
- **Deducciones aplicables:** Familias numerosas, discapacidad y otros factores.
- **Límites de patrimonio:** Denegación si se superan valores catastrales establecidos.

## 4. Tipos de Becas y Cuantías
### Universitarios
- **Beca de matrícula:** Cubierta en universidades públicas para créditos de primera matrícula.
- **Beca de cuantía fija:** Incluye ayudas por renta, residencia y excelencia académica.
- **Beca de cuantía variable:** Según nota media y renta familiar.
- **Ayudas adicionales:** 50% extra para estudiantes con discapacidad, reducción del 50% si la matrícula es en un solo cuatrimestre.

### No Universitarios
- **Beca básica:** 350€ para ciclos formativos de grado básico.
- **Beca de cuantía variable:** Mínimo de 60€.
- **Ayudas de residencia:** Para estudiantes sin centro educativo en su localidad.
- **Beca adicional de 500€:** Para estudiantes de familias monoparentales.

## 5. Ayudas Adicionales
- **Transporte en islas, Ceuta y Melilla:** Entre 442€ y 937€, según el trayecto.
- **Incrementos específicos para FP en Canarias:** 300€ adicionales.






### DeepSeek

## 1. Requisitos Económicos
Umbrales de Renta Familiar:

      Miembros familia	Umbral 1 (€)	Umbral 2 (€)	Umbral 3 (€)
      1	8,843	13,898	14,818
      2	13,264	23,724	25,293
      3	17,685	32,201	34,332
      4	22,107	38,242	40,773
      5	25,644	42,743	45,572
      6	29,181	46,142	49,196
      7	32,718	49,503	52,780
      8	36,255	52,850	56,348
      Cada adicional	+3,536	+3,340	+3,561

### Límites de Patrimonio:

- Propiedades urbanas: 42,900€ (excluida vivienda habitual)

- Coeficientes reductores según año revisión catastral (0.25 a 0.50)

- Construcciones rústicas: 42,900€

- Fincas rústicas: 13,130€ por miembro computable

- Rendimientos capital: 1,700€ máximo anual

## 2. Becas Universitarias
### Cuantías:
- Matrícula: 100% créditos primera matrícula (precios públicos)

- Fija renta: 1,700€ (requiere Umbral 1)

- Residencia: 2,500€ (requiere Umbral 2 + justificación)

- Excelencia académica:

      Nota media	Cuantía (€)
      8.0-8.49	50
      8.5-8.99	75
      9.0-9.49	100
      ≥9.5	125

### Requisitos Académicos:
### Créditos mínimos:

  - Tiempo completo: 60 créditos

  - Parcial: 30-59 créditos (beca reducida)

  - Rendimiento previo:

        Rama de estudio	% créditos a superar
        Artes/Humanidades	90%
        Ciencias	65%
        Ciencias Salud	80%
        Ingenierías	65%

## 3. Becas No Universitarias
### Cuantías
- Cuantías Básicas:
      FP Básica: 350€
      Resto enseñanzas: 300€

- Variable: Mínimo 60€ (según renta y rendimiento)

- Complementos Territoriales:
      Situación	Cuantía (€)
      Islas/CEUTA-MELILLA	442
      Islas menores*	623
      Desplazamiento peninsular	888-937
      FP + movilidad insular	+300
      *Lanzarote, Fuerteventura, La Gomera, El Hierro, La Palma, Menorca, Ibiza, Formentera

### Requisitos por Enseñanza:
      FP Grado Medio/Superior:
      500 horas mínimas de matrícula
      85% módulos aprobados curso anterior

      Bachillerato:
      4 asignaturas mínimas
      Aprobar todas menos 1

## 4. Deducciones Especiales
### Sobre Renta Familiar:
- Discapacidad:

      ≥33%: 1,811€
      ≥65%: 2,881€ (4,000€ si es el solicitante)
      Hermanos universitarios: 1,176€ c/u
      Familias numerosas:
      General: 525€
      Especial: 800€
      Monoparentales: 500€

## 5. Condiciones Especiales
### Para Discapacitados (≥65%):
- Reducción carga lectiva hasta 50%

- Incremento del 50% en cuantías fijas

- Exención requisitos mínimos créditos

### Movilidad Internacional:
- UE: 800€ para prácticas FP

### Requisitos:

- Duración mínima: 1 cuatrimestre

- Formar parte del plan de estudios

## 6. Plazos y Gestión
### Calendario:
- Presentación: 19/03 a 10/05/2024

- Excepciones: Hasta 31/12 por:

      Fallecimiento sustentador
      Jubilación forzosa

### Documentación:
- Obligatoria:

  - Autorización consulta datos

  - Justificación domicilio

  - Certificado discapacidad (si aplica)

- Específica:

  - Contratos alquiler (para residencia)

  - Títulos familia numerosa

ChatGPT add some adittional information like other scholarships, but it lacks the percentages of credits passed necessaries in the university level depending on the branch of knowledge. Overall, both models offer a solid summary of the key information relevant to a scholarship applicant.